In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
import requests

# Initialize the language model
llm = ChatOpenAI(temperature=0.1)

# Download the document from the Gist URL
response = requests.get("https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw")
with open("document.txt", "w", encoding="utf-8") as f:
    f.write(response.text)

# Load the document from the downloaded file
loader = TextLoader("document.txt", encoding="utf-8")

# Split the document into overlapping chunks
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=splitter)

# Set up cached OpenAI embeddings
embeddings = OpenAIEmbeddings()
cache_dir = LocalFileStore("./.cache/")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# Store document chunks in a FAISS vectorstore
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

# Set up conversation memory to keep chat history
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

# Create a StuffDocuments RAG chain with memory
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    output_key="answer"
)

# Ask questions to the chain
questions = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?"
]

for q in questions:
    result = qa_chain.invoke({"question": q})
    print(f"\n❓ {q}\n👉 {result['answer']}")


❓ Is Aaronson guilty?
👉 In the text provided, it is mentioned that Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with. However, it is also stated that Winston had never seen the photograph that disproved their guilt and that it had never existed, he had invented it. This ambiguity suggests that the guilt of Aaronson and the others may not be as clear-cut as it seems.

❓ What message did he write in the table?
👉 He wrote "FREEDOM IS SLAVERY" and then wrote "TWO AND TWO MAKE FIVE" beneath it.

❓ Who is Julia?
👉 Julia is a character from George Orwell's novel "1984." She is a woman who becomes romantically involved with the protagonist, Winston Smith. Their relationship is a central part of the story, as they rebel against the oppressive regime of the Party in the dystopian society depicted in the novel.
